In [1]:
from qiskit import QuantumCircuit, execute
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit_aer.noise import NoiseModel, QuantumError, pauli_error
from qiskit.quantum_info import Operator, PTM, Pauli
import numpy as np
import random

In [2]:
id_gate = Operator(np.eye(4))


def noisy_mcm(circ, cbit):
    """Noisy mid-circuit measurement.
    The measurement is in the measure-and-prepare fashion:
    We apply a Pauli noise, then an ideal mid-circuit measurement, followed by another Pauli noise.
    """
    circ.cnot(0, 1)
    circ.unitary(id_gate, [0, 1], label='pre')
    circ.measure(1, cbit)
    circ.unitary(id_gate, [0, 1], label='post')

    
def noisy_sp(circ, p):
    """Noisy state preparation.
    Though almost any initial state works for our protocol, it would be better for the initial state to big overlap with the ideal operator.
    Here we assume that we prepare the eigen state of the ideal operator and then apply a special Pauli noise (same as the terminating measurement nosie).
    """
    p = p[::-1]
    for i in range(2):
        if p[i] == 'I':
            p = p[:i] + random.choice(['X', 'Y', 'Z']) + p[i + 1:]
        if p[i] == 'X':
            circ.h(i)
        elif p[i] == 'Y':
            circ.h(i)
            circ.s(i)
    circ.unitary(id_gate, [0, 1], label='spam')
    

def noisy_m(circ, p, cbit):
    """Noisy terminating measurement.
    We apply a special Pauli noise and then make an ideal terminating measurement.
    If the measured operator is I, then we will still make a measurement but will ignore the output.
    """
    p = p[::-1]
    circ.unitary(id_gate, [0, 1], label='spam')
    for i in range(2):
        if p[i] == 'I':
            p = p[:i] + random.choice(['X', 'Y', 'Z']) + p[i + 1:]
        if p[i] == 'X':
            circ.h(i)
        elif p[i] == 'Y':
            circ.sdg(i)
            circ.h(i)
    circ.measure([0, 1], [cbit, cbit + 1])

In [3]:
"""
The noise model.
"""
rate_pre = {}
for i in ["I", "X", "Y", "Z"]:
    for j in ["I", "X", "Y", "Z"]:
        rate_pre[i+j] = np.random.rand() * 0.01
rate_pre["II"] += 1 - np.sum([v for v in rate_pre.values()])
err_pre = pauli_error(list(rate_pre.items()))
fid_pre = np.real_if_close(np.diag(PTM(err_pre).data))

rate_post = {}
for i in ["I", "X", "Y", "Z"]:
    for j in ["I", "X", "Y", "Z"]:
        rate_post[i+j] = np.random.rand() * 0.02
rate_post["II"] += 1 - np.sum([v for v in rate_post.values()])
err_post = pauli_error(list(rate_post.items()))
fid_post = np.real_if_close(np.diag(PTM(err_post).data))

rate_spam = {}
for i in [["I"], ["X", "Y", "Z"]]:
    for j in [["I"], ["X", "Y", "Z"]]:
        rate = (np.random.rand() + len(i) + len(j)) * 0.005 # to make unlearnable info obviously wrong
        for ii in i:
            for jj in j:
                rate_spam[ii + jj] = rate
rate_spam["II"] += 1 - np.sum([v for v in rate_spam.values()])
err_spam = pauli_error(list(rate_spam.items()))

noise_model = NoiseModel()
noise_model.add_basis_gates(['unitary'])
noise_model.add_all_qubit_quantum_error(err_pre, 'pre')
noise_model.add_all_qubit_quantum_error(err_post, 'post')
noise_model.add_all_qubit_quantum_error(err_spam, 'spam')
backend_noisy = AerSimulator(noise_model=noise_model)

In [4]:
def fid2st(fid):
    s = fid[0]
    t = fid[0]
    if fid[1] == '0':
        s = 'I' + s
    else:
        s = 'Z' + s
    if fid[2] == '0':
        t = 'I' + t
    else:
        t = 'Z' + t
    cn_circ = QuantumCircuit(2, 0)
    cn_circ.cnot(0, 1)
    s = Pauli(s).evolve(cn_circ, frame='h').to_label()
    sgn = 1
    if s[0] == '-':
        sgn *= -1
        s = s[1:]
    return sgn, s, t

In [57]:
def learn_single_fid(fid, shots=10000):
    """Learn e+\partial e"""
    sgn, s, t = fid2st(fid)
    circ = QuantumCircuit(2, 3)
    noisy_sp(circ, s)
    noisy_mcm(circ, 0)
    noisy_m(circ, t, 1)
    counts = execute(circ, backend_noisy, shots=shots).result().get_counts()
    s_avg = 0
    for res in counts:
        count = counts[res]
        res_par = np.array([int(c) for c in res])[::-1]
        for i in range(2):
            if t[i] == 'I':
                res_par[2 - i] = 0
        s_avg += sgn * count * (-1) ** (res_par[0] * (int(fid[1]) + int(fid[2])) + res_par[1] + res_par[2])
    s_avg /= shots
    
    circ2 = QuantumCircuit(2, 2)
    noisy_sp(circ2, s)
    noisy_m(circ2, s, 0)
    counts = execute(circ2, backend_noisy, shots=shots).result().get_counts()
    t_avg = 0
    for res in counts:
        count = counts[res]
        res_par = np.array([int(c) for c in res])[::-1]
        for i in range(2):
            if s[i] == 'I':
                res_par[1 - i] = 0
        t_avg += count * (-1) ** (res_par[0] + res_par[1])
    t_avg /= shots
    return np.log(s_avg / t_avg)

In [6]:
learned_fid = {}
for i in ["I", "X", "Y", "Z"]:
    for j in ["0", "1"]:
        for k in ["0", "1"]:
            learned_fid[i + j + k] = learn_single_fid(i + j + k, shots=100000)

In [7]:
# lemma 5
learned_fid["Z00"]+learned_fid["Z11"]-learned_fid["Z01"]-learned_fid["Z10"]

0.003984899823260057

In [8]:
def str2idx(p): # II, IX, IY, IZ, XI, XX...
    ret = 0
    for i in range(2):
        if p[i] == "X":
            ret += 1 * 4 ** (1 - i)
        elif p[i] == "Y":
            ret += 2 * 4 ** (1 - i)
        elif p[i] == "Z":
            ret += 3 * 4 ** (1 - i)
    return ret

In [80]:
true_fid = {}
for i in ["I", "X", "Y", "Z"]:
    for j in ["0", "1"]:
        for k in ["0", "1"]:
            _, s, t = fid2st(i + j + k)
            true_fid[i + j + k] = fid_pre[str2idx(s)] * fid_post[str2idx(t)]

In [10]:
# self loop is learnable
print(np.exp(learned_fid["Y01"]))
print(np.real_if_close(true_fid["Y01"]))

0.8059971323483572
0.8042725608862727


In [11]:
# cycle is learnable
print(np.exp(learned_fid["Z01"] + learned_fid["X00"]))
print(np.real_if_close(true_fid["Z01"] * true_fid["X00"]))

0.5708322676213492
0.5804743120670213


In [12]:
# simple edge is not learnable
print(np.exp(learned_fid["Z01"]))
print(np.real_if_close(true_fid["Z01"]))

0.7965055762081784
0.7485210658487629


In [16]:
def noiseless_concat(circ, t, s):
    for i in range(2):
        if t[i] != s[i]:
            if t[i] == 'I' or s[i] == 'I':
                raise ValueError('Mismatched Pauli pattern.')
            if t[i] == 'X':
                if s[i] == 'Y':
                    circ.s(1 - i)
                else:
                    circ.h(1 - i)
            elif t[i] == 'Y':
                if s[i] == 'Z':
                    circ.sdg(1 - i)
                    circ.h(1 - i)
                else:
                    circ.sdg(1 - i)
            else:
                if s[i] == 'X':
                    circ.h(1 - i)
                else:
                    circ.h(1 - i)
                    circ.s(1 - i)

In [56]:
def learn_multiple_fid(fids, shots=10000):
    """Learn a path"""
    l = len(fids)
    ss = [""] * l
    ts = [""] * l
    tot_sgn = 1
    for i in range(l):
        sgn, ss[i], ts[i] = fid2st(fids[i])
        tot_sgn *= sgn
    circ = QuantumCircuit(2, l + 2)
    noisy_sp(circ, ss[0])
    for i in range(l - 1):
        noisy_mcm(circ, i)
        noiseless_concat(circ, ts[i], ss[i + 1])
    noisy_mcm(circ, l - 1)
    noisy_m(circ, ts[l - 1], l)
    counts = execute(circ, backend_noisy, shots=shots).result().get_counts()
    s_avg = 0
    for res in counts:
        count = counts[res]
        res_par = np.array([int(c) for c in res])[::-1]
        for i in range(2):
            if ts[l - 1][i] == 'I':
                res_par[l + 1 - i] = 0
        mxy = 0
        for i in range(l):
            mxy += res_par[i] * (int(fids[i][1]) + int(fids[i][2]))
        s_avg += tot_sgn * count * (-1) ** (mxy + res_par[l] + res_par[l + 1])
    s_avg /= shots
    
    circ2 = QuantumCircuit(2, 2)
    noisy_sp(circ2, ss[0])
    noisy_m(circ2, ss[0], 0)
    counts = execute(circ2, backend_noisy, shots=shots).result().get_counts()
    t_avg = 0
    for res in counts:
        count = counts[res]
        res_par = np.array([int(c) for c in res])[::-1]
        for i in range(2):
            if ss[0][i] == 'I':
                res_par[1 - i] = 0
        t_avg += count * (-1) ** (res_par[0] + res_par[1])
    t_avg /= shots
    return np.log(s_avg / t_avg)

In [76]:
targ = ['X11','Y11', 'Y00', 'Z01', 'X00', 'Z00', 'Z01', 'I11', 'Z11']
learned_res = learn_multiple_fid(targ, shots=100000) 
print(np.exp(learned_res)) # learned value of this product
print(np.real_if_close(np.prod([true_fid[i] for i in targ]))) # true value of this product
targ = ['Y01','X10', 'Z01', 'X01', 'Y10', 'Z01', 'I10', 'I01', 'Z10', 'Z01']
learned_res -= learn_multiple_fid(targ, shots=100000)
print(learned_res / 16) # learned p^I_{11}

0.10759375583058647
0.10143446022410743
-0.0022787542950186424


In [81]:
pI11 = 0.
for k, v in true_fid.items():
    pI11 += v * (-1) ** (int(k[1]) + int(k[2]))
pI11 /= 16
print(pI11) # true value of p^I_{00}

0.0007234988162280395


In [82]:
pI11_apx = 0.
for k, v in true_fid.items():
    pI11_apx += (1 + np.log(v)) * (-1) ** (int(k[1]) + int(k[2]))
pI11_apx /= 16
print(pI11_apx) # under first order approximation p^I_{00}=0 (lemma 5)

6.938893903907228e-18


In [84]:
targ = ['I00', 'I01', 'Z11', 'I11', 'Z10', 'Z00', 'Z01', 'I10']
print(learn_multiple_fid(targ, shots=100000) / 8 + 1) # learned value of p^I_{00} + p^Z_{00}

0.8033738055224313


In [86]:
pI00Z00 = 0
for k, v in true_fid.items():
    if k[0] == 'I' or k[0] == 'Z':
        pI00Z00 += v
pI00Z00 /= 8
print(pI00Z00) # true value of p^I_{00} + p^Z_{00}

0.8249444717056876


In [87]:
pI00Z00_apx = 0
for k, v in true_fid.items():
    if k[0] == 'I' or k[0] == 'Z':
        pI00Z00_apx += 1 + np.log(v)
pI00Z00_apx /= 8
print(pI00Z00_apx) # p^I_{00} + p^Z_{00} under first order approximation, one can see that error mostly come from the approximation

0.8018906890961874
